<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Google Sheets to Plugin Pipeline

**Tags:** #gsheet #pipeline #automation #snippet #avatar #openai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

## Input

### Import libraries

In [ ]:
import naas_data_product
from naas.pipeline import Pipeline, NotebookStep, End
import naas

### Setup variables

In [ ]:
# Avatar meta
avatar_name = "Jeremy Ravenel"
avatar_image = "https://cl-151.imagineapi.dev/assets/0954e7c3-c40c-4baf-91ea-20a592417105.png"
avatar_model_alias = "naas/jeremy_linkedin_posts"
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "ai-characters", avatar_name.lower().replace(" ", "_"), avatar_model_alias.replace("/", "_"))
print("Output directory:", output_dir)

# OpenAI
openai_api_key = naas.secret.get("OPENAI_API_KEY")

# Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1wediMdHcq5WDqLMZ7ryNrcPxCmlX8BX4ZEl3JNWT8wg/edit#gid=0"
sheet_name = "Jeremy"

## Model

### Setup notebooks
This section is made to declare all the notebooks and create unique IDs that will be used in the pipeline. 

In [ ]:
gsheet = NotebookStep(
    name='Google Sheets', #notebook name display in pipeline
    notebook_path="fine_tuning/Google_Sheet_Fine_tune_dataset_with_OpenAI.ipynb", # notebook path to be run
    parameters={
        "avatar_name": avatar_name,
        "output_dir": output_dir,
        "spreadsheet_url": spreadsheet_url,
        "sheet_name": sheet_name,
        "openai_api_key": openai_api_key,        
    } #inserted in your notebook after cell tagged 'parameters'
)

supabase = NotebookStep(
    name='Supabase', #notebook name display in pipeline
    notebook_path="fine_tuning/Supabase_Update_available_models.ipynb", # notebook path to be run
    parameters={
        "avatar_name": avatar_name,
        "output_dir": output_dir,
        "avatar_model_alias": avatar_model_alias,
    } #inserted in your notebook after cell tagged 'parameters'
)

plugin = NotebookStep(
    name='Plugin', #notebook name display in pipeline
    notebook_path="fine_tuning/OpenAI_Act_as_avatar.ipynb", # notebook path to be run
    parameters={
        "avatar_name": avatar_name,
        "avatar_image": avatar_image,
        "output_dir": output_dir,
    } #inserted in your notebook after cell tagged 'parameters'
)

## Output

### Run pipeline

In [ ]:
pipeline = Pipeline()

pipeline >> gsheet >> supabase >> plugin >> End()

pipeline.run(outputs_path=os.path.join(output_dir, "pipeline_executions"))